# Загружаем данные

по результатам предсказаний instance

адрес и камера айди photos_test_info

хвосты tails

цвета colors

идеальный вариант etalons - получен с помощью https://github.com/AlexanderNevarko/pet-search/blob/main/notebooks/EDA.ipynb

In [1]:
import pandas as pd
import numpy as np

gt = pd.read_csv('test')
predicted = gt.copy()

instance = pd.read_csv('instance')
address_cam_id = pd.read_csv('photos_test_info.csv')
tails = pd.read_csv('tails_test.csv')
colors = pd.read_csv('colors_test.csv')
etalons = pd.read_csv('etalons')

In [2]:
address_cam_id['filename'] = address_cam_id['file_path'].apply(lambda x: str(x).split('/')[-1])

In [3]:
assert (gt.shape[0] == address_cam_id.shape[0]), 'sizes not equal'

# Поскольку у нас нет правильных ответов для адреса, устанавливаем адрес и для эталона

In [4]:
for x, i in enumerate(address_cam_id.filename):
    ind = predicted[predicted.filename == i].index[0]
    predicted.loc[ind, 'address'] = address_cam_id.loc[x, 'street_address']
    predicted.loc[ind, 'cam_id'] = address_cam_id.loc[x, 'camera_id']
for x, i in enumerate(address_cam_id.filename):
    ind = etalons[etalons.filename == i].index[0]
    etalons.loc[ind, 'address'] = address_cam_id.loc[x, 'street_address']
    etalons.loc[ind, 'cam_id'] = address_cam_id.loc[x, 'camera_id']

In [5]:
for x, i in enumerate(tails.filename):
    ind = predicted[predicted.filename == i].index[0]
    predicted.loc[ind, 'tail'] = tails.loc[x, 'tail']

In [6]:
for x, i in enumerate(colors.filename):
    ind = predicted[predicted.filename == i].index[0]
    predicted.loc[ind, 'color'] = colors.loc[x, 'color']

In [7]:
for x, i in enumerate(instance.filename):
    ind = predicted[predicted.filename == i].index[0]
    predicted.loc[ind, 'is_animal_there'] = instance.loc[x, 'is_animal_there']
    predicted.loc[ind, 'is_it_a_dog'] = instance.loc[x, 'is_it_a_dog']
    predicted.loc[ind, 'is_the_owner_there'] = instance.loc[x, 'is_the_owner_there']

In [8]:
#predicted['address'] = predicted['address'].replace('NONE', '')
#predicted['cam_id'] = predicted['cam_id'].replace('NONE', '')

#etalons['address'] = etalons['address'].replace('NONE', '')
#etalons['cam_id'] = etalons['cam_id'].replace('NONE', '')

# Берем итоговый скор от экспертов

In [9]:
def score_predictions(pred_df, real_df):
    classification_weights = np.array([0.1, 0.1, 0.2, 0.4, 0.2])
    classification_columns = [
        "is_animal_there", "is_it_a_dog", "is_the_owner_there", "color", "tail"
    ]
    Y_hat_c = pred_df[classification_columns].fillna("")
    Y_c = real_df[classification_columns].fillna("")
    sums = Y_hat_c[classification_columns[1:]].sum(1).values > 0
    classification_score = 0
    if not np.logical_and(sums, Y_hat_c["is_animal_there"].values == 0).any():
        sums2 = Y_hat_c[classification_columns[1:]].sum(1).values > 0
        if not np.logical_and(sums2, Y_hat_c["is_it_a_dog"].values == 0).any():
            classification_score = np.sum(
                classification_weights*np.array(
                    [
                        balanced_accuracy_score(
                            y_pred=Y_hat_c[a].values, y_true=Y_c[a].values
                        ) 
                             for a in classification_columns
                    ]
                )
            )
    return(classification_score*100)

def score_texts(pred_df, real_df):
    text_columns = ["address", "cam_id"]
    Y_hat_t = pred_df[text_columns].fillna("")
    Y_t = real_df[text_columns].fillna("")
    Y_hat_t["cam_id"] = Y_hat_t["cam_id"].apply(str)
    Y_t["cam_id"] = Y_t["cam_id"].apply(str)
    ad_lens = Y_hat_t["address"].apply(len).values
    cid_lens = Y_hat_t["cam_id"].apply(len).values
    largest_lengths = np.array(list(zip(max(a,b) for a,b in zip(ad_lens, cid_lens))))
    ad_levs = np.array(
        list(zip(distance(a, b) for a,b in zip(Y_hat_t["address"], Y_t["address"])))
    )
    cid_levs = np.array(
        list(zip(distance(a, b) for a,b in zip(Y_hat_t["cam_id"], Y_t["cam_id"])))
    )
    ad_levs = ad_levs/largest_lengths
    cid_levs = cid_levs/largest_lengths
    text_score = np.mean(np.array([1/(min(a,b)+10**-5) for a,b in zip(ad_levs, cid_levs)]))
    return(text_score)

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from Levenshtein import distance

In [11]:
your_solution = predicted
true_solution = etalons

In [12]:
your_solution.tail()

,directory,filename,is_animal_there,is_it_a_dog,is_the_owner_there,color,tail,address,cam_id
309,Хоязин и собака,А47.jpg,1.0,1.0,1.0,1.0,0.0,"город Москва, Шелепихинская набережная, дом 22",PVN_hd_TSAO_4915_2
310,Хоязин и собака,А84.jpg,1.0,1.0,1.0,1.0,0.0,"город Москва, улица Каховка, дом 5, корпус 4",PVN_hd_UZAO_7277_1
311,Хоязин и собака,С76.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, Левобережная улица, дом 4, корпу...",PVN_hd_SAO_6159_54
312,Хоязин и собака,С63.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, Центросоюзный переулок, дом 4",PVN_hd_TSAO_3647_1
313,Хоязин и собака,С88.jpg,1.0,1.0,1.0,2.0,0.0,"город Москва, Центросоюзный переулок, дом 4",PVN_hd_TSAO_3647_1


In [13]:
true_solution.tail()

,directory,filename,is_animal_there,is_it_a_dog,is_the_owner_there,color,tail,address,cam_id
309,Хоязин и собака,А47.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, Шелепихинская набережная, дом 22",PVN_hd_TSAO_4915_2
310,Хоязин и собака,А84.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, улица Каховка, дом 5, корпус 4",PVN_hd_UZAO_7277_1
311,Хоязин и собака,С76.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, Левобережная улица, дом 4, корпу...",PVN_hd_SAO_6159_54
312,Хоязин и собака,С63.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, Центросоюзный переулок, дом 4",PVN_hd_TSAO_3647_1
313,Хоязин и собака,С88.jpg,1.0,1.0,1.0,0.0,0.0,"город Москва, Центросоюзный переулок, дом 4",PVN_hd_TSAO_3647_1


# Итоговые результаты

In [14]:
#@title Запустите эту клетку, чтобы получить результат

print(
  "Оценка классификации", score_predictions(your_solution, true_solution)
)
print(
  "Оценка предсказания текста", 
  score_texts(your_solution, your_solution)
)

Оценка классификации 84.59043585501963
Оценка предсказания текста 99999.99999999999
